## **Info**

* https://www.kaggle.com/datasets/rushikeshdarge/constitution-of-india

# **Set Up**

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import plotly.express as px

import os
from zipfile import ZipFile

In [18]:
import re
import nltk
from nltk.tag import pos_tag
from collections import defaultdict,Counter
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize,sent_tokenize
from nltk.corpus import stopwords as sw
import string
from nltk.corpus import wordnet as wn

[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1002)>
[nltk_data] Error loading punkt: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1002)>
[nltk_data] Error loading averaged_perceptron_tagger: <urlopen error
[nltk_data]     [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify
[nltk_data]     failed: unable to get local issuer certificate
[nltk_data]     (_ssl.c:1002)>
[nltk_data] Error loading wordnet: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1002)>


In [20]:
from psutil import *

# CPU Specification
!lscpu |grep 'Model name'
print('CPU Core Count:',cpu_count())

# GPU Specification
!nvidia-smi -L

# System Memory
!free -h --si | awk '/Mem:/{print $2}'

# System Storage Memory
!df -h / |awk '{print $4}'

zsh:1: command not found: lscpu
CPU Core Count: 8
zsh:1: command not found: nvidia-smi
zsh:1: command not found: free
Avail
245Gi


# **Data Import**

In [2]:
!kaggle datasets download -d rushikeshdarge/constitution-of-india

constitution-of-india.zip: Skipping, found more recently modified local copy (use --force to force download)


In [3]:
with ZipFile("constitution-of-india.zip", "r") as zip_ref:
    # Get list of file names in zip
    list_of_files = zip_ref.namelist()
    print('No. Of Files In Zip File: ',len(list_of_files),'\n')
    print(list_of_files)

No. Of Files In Zip File:  2 

['Constitution Of India.csv', 'Index.csv']


In [4]:
# Extract Zip File Contents In Main Directory
with ZipFile("constitution-of-india.zip", "r") as zip_ref:
    # Get list of file names in zip
    list_of_files = zip_ref.namelist()

    for file in list_of_files:
        file_content = zip_ref.read(file)
        with open(os.path.basename(file), "wb") as f:
                f.write(file_content)

In [5]:
# Read CSV File As Dataframe
df = pd.read_csv('Constitution Of India.csv')
print(df.shape)
df.head()

(456, 1)


,Articles
0,"1. Name and territory of the Union\n(1) India,..."
1,1. The territories of the States; the Union te...
2,2. Admission or establishment of new States: P...
3,2A. Sikkim to be associated with the Union Rep...
4,3. Formation of new States and alteration of a...


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 456 entries, 0 to 455
Data columns (total 1 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Articles  456 non-null    object
dtypes: object(1)
memory usage: 3.7+ KB


In [7]:
df.rename(columns={'Articles':'Text'},inplace=True)
df.head()

,Text
0,"1. Name and territory of the Union\n(1) India,..."
1,1. The territories of the States; the Union te...
2,2. Admission or establishment of new States: P...
3,2A. Sikkim to be associated with the Union Rep...
4,3. Formation of new States and alteration of a...


# **Data Cleaning**

In [8]:
df['Word_Count_00'] = df['Text'].apply(lambda x: len(x))

In [9]:
df.head()

,Text,Word_Count_00
0,"1. Name and territory of the Union\n(1) India,...",217
1,1. The territories of the States; the Union te...,134
2,2. Admission or establishment of new States: P...,160
3,2A. Sikkim to be associated with the Union Rep...,129
4,3. Formation of new States and alteration of a...,1302


In [10]:
df.sort_values('Word_Count_00',ascending=False)

,Text,Word_Count_00
437,371D. Special provisions with respect to the s...,7256
428,"366. Definition In this Constitution, unless t...",6961
434,371A. Special provision with respect to the St...,6396
439,371F. Special provisions with respect to the S...,5884
412,352. Proclamation of Emergency\n(1) If the Pre...,5046
...,...,...
245,232. Omitted by the Constitution 7 th Amendmen...,87
453,393. Short title This Constitution may be call...,74
70,63. The Vice President of India There shall be...,72
224,214. High Courts for States There shall be a H...,70


## **Tokenize**

In [15]:
def preprocess(text):
    lemmatizer = nltk.WordNetLemmatizer().lemmatize
    text = re.sub('\W+', ' ', str(text))
    text = re.sub(r'[0-9]+', '', text.lower())
    word_pos = nltk.pos_tag(nltk.word_tokenize(text))
    normalized_text_lst = [lemmatizer(x[0], get_wordnet_pos(x[1])).lower() for x in word_pos]
    stop_words_free = [i for i in normalized_text_lst if i not in english_stopwords and len(i) > 3]
    stop_words_free = list(set(stop_words_free))
    return(stop_words_free)

In [19]:
df['Text_01'] = df['Text'].apply(preprocess)

LookupError: 
**********************************************************************
  Resource [93mpunkt[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt/PY3/english.pickle[0m

  Searched in:
    - '/Users/akshayyewle/nltk_data'
    - '/Library/Frameworks/Python.framework/Versions/3.11/nltk_data'
    - '/Library/Frameworks/Python.framework/Versions/3.11/share/nltk_data'
    - '/Library/Frameworks/Python.framework/Versions/3.11/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
    - ''
**********************************************************************


## **Stopword Removal**

In [11]:
from nltk.corpus import stopwords

df['Text_01'] = df['Text'].apply(lambda x: )

# **Extract Text**

In [12]:
Text = ''

Text + 'Akshay'

'Akshay'

In [13]:
Text

''

In [14]:
Text = ''

for i in df.index:
    Text.

SyntaxError: invalid syntax (328302112.py, line 4)